# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

---

_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [1]:
!pip3 install -U sagemaker

  Using cached sagemaker-2.251.1-py3-none-any.whl.metadata (17 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached boto3-1.40.40-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.40-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
Using cached sagemaker-2.251.1-py3-none-any.whl (1.7 MB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached boto3-1.40.40-py3-none-any.whl (139 kB)
Using cached botocore-1.40.40-py3-none-any.whl (14.0 MB)
Using cached s3transfer-0.14.0-py3-none-any.whl (85 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.1
    Uninstalling botocore-1.37.1:
      Successfully uninstalled botocore-1.37.1
  Attempting uninstall: s3transferm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [botocore]
  

In [2]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [3]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
94,862028,M,15.060,19.83,100.30,705.6,0.10390,0.15530,0.17000,0.08815,...,18.23,24.23,123.50,1025.0,0.1551,0.4203,0.52030,0.21150,0.2834,0.08234
566,926954,M,16.600,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.98,34.12,126.70,1124.0,0.1139,0.3094,0.34030,0.14180,0.2218,0.07820
158,871122,B,12.060,12.74,76.84,448.6,0.09311,0.05241,0.01972,0.01963,...,13.14,18.41,84.08,532.8,0.1275,0.1232,0.08636,0.07025,0.2514,0.07898
530,91858,B,11.750,17.56,75.89,422.9,0.10730,0.09713,0.05282,0.04440,...,13.50,27.98,88.52,552.3,0.1349,0.1854,0.13660,0.10100,0.2478,0.07757
391,903483,B,8.734,16.84,55.27,234.3,0.10390,0.07428,0.00000,0.00000,...,10.17,22.80,64.01,317.0,0.1460,0.1310,0.00000,0.00000,0.2445,0.08865
357,901028,B,13.870,16.21,88.52,593.7,0.08743,0.05492,0.01502,0.02088,...,15.11,25.58,96.74,694.4,0.1153,0.1008,0.05285,0.05556,0.2362,0.07113
17,84862001,M,16.130,20.68,108.10,798.8,0.11700,0.20220,0.17220,0.10280,...,20.96,31.48,136.80,1315.0,0.1789,0.4233,0.47840,0.20730,0.3706,0.11420
536,91979701,M,14.270,22.55,93.77,629.8,0.10380,0.11540,0.14630,0.06139,...,15.29,34.27,104.30,728.3,0.1380,0.2733,0.42340,0.13620,0.2698,0.08351


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [4]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
113,864292,0,10.510,20.19,68.64,334.2,0.11220,0.13030,0.06476,0.03068,...,11.160,22.75,72.62,374.4,0.1300,0.2049,0.1295,0.06136,0.2383,0.09026
209,8810436,0,15.270,12.91,98.17,725.5,0.08182,0.06230,0.05892,0.03157,...,17.380,15.92,113.70,932.7,0.1222,0.2186,0.2962,0.10350,0.2320,0.07474
86,86135501,1,14.480,21.46,94.25,648.2,0.09444,0.09947,0.12040,0.04938,...,16.210,29.25,108.40,808.9,0.1306,0.1976,0.3349,0.12250,0.3020,0.06846
242,883852,0,11.300,18.19,73.93,389.4,0.09592,0.13250,0.15480,0.02854,...,12.580,27.96,87.16,472.9,0.1347,0.4848,0.7436,0.12180,0.3308,0.12970
363,9010872,0,16.500,18.29,106.60,838.1,0.09686,0.08468,0.05862,0.04835,...,18.130,25.45,117.20,1009.0,0.1338,0.1679,0.1663,0.09123,0.2394,0.06469
525,91805,0,8.571,13.10,54.53,221.3,0.10360,0.07632,0.02565,0.01510,...,9.473,18.45,63.30,275.6,0.1641,0.2235,0.1754,0.08512,0.2983,0.10490
24,852552,1,16.650,21.38,110.00,904.6,0.11210,0.14570,0.15250,0.09170,...,26.460,31.56,177.00,2215.0,0.1805,0.3578,0.4695,0.20950,0.3613,0.09564
180,873592,1,27.220,21.87,182.10,2250.0,0.10940,0.19140,0.28710,0.18780,...,33.120,32.85,220.80,3216.0,0.1472,0.4034,0.5340,0.26880,0.2856,0.08082


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [5]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

Let's upload those data sets in S3

In [6]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-851725636446/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [7]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2025-09-29-17-11-41


2025-09-29 17:11:42 Starting - Starting the training job...
2025-09-29 17:12:17 Downloading - Downloading input data...
2025-09-29 17:12:37 Downloading - Downloading the training image......
2025-09-29 17:13:33 Training - Training image download completed. Training in progress../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-29 17:13:44.130 ip-10-2-207-251.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-09-29 17:13:44.191 ip-10-2-207-251.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-09-29:17:13:44:INFO] Imported framework sagemaker_xgboost_container.training
[2025-09-29:17:13:44:INFO] Failed to parse hyperparameter objective

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [8]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m5.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-09-29-17-14-59-067
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-09-29-17-14-59-753


.............................../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-29:17:20:12:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:20:12:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:20:12:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|inv

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [9]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [10]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.956720
1,0.952197
2,0.730796
3,0.994905
4,0.956720
5,0.964800
6,0.004089
7,0.987876


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [11]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-09-29-17-21-04-271


.............................../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-29:17:26:09:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:26:09:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:26:09:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package o

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [12]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.66560,0.71190,0.26540,0.4601,0.11890,0.956720
1,844981,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,30.73,106.20,739.3,0.1703,0.54010,0.53900,0.20600,0.4378,0.10720,0.952197
2,846381,15.85,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,...,27.66,112.00,876.5,0.1131,0.19240,0.23220,0.11190,0.2809,0.06287,0.730796
3,849014,19.81,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.09498,0.1582,...,30.88,186.80,2398.0,0.1512,0.31500,0.53720,0.23880,0.2768,0.07615,0.994905
4,8511133,15.34,14.26,102.50,704.4,0.10730,0.21350,0.20770,0.09756,0.2521,...,19.08,125.10,980.9,0.1390,0.59540,0.63050,0.23930,0.4667,0.09946,0.956720
5,856106,13.28,20.28,87.32,545.2,0.10410,0.14360,0.09847,0.06158,0.1974,...,28.00,113.10,907.2,0.1530,0.37240,0.36640,0.14920,0.3739,0.10270,0.964800
6,857374,11.94,18.24,75.71,437.6,0.08261,0.04751,0.01972,0.01349,0.1868,...,21.33,83.67,527.2,0.1144,0.08906,0.09203,0.06296,0.2785,0.07408,0.004089
7,85922302,12.68,23.84,82.69,499.0,0.11220,0.12620,0.11280,0.06873,0.1905,...,33.47,111.80,888.3,0.1851,0.40610,0.40240,0.17160,0.3383,0.10310,0.987876


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [13]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-09-29-17-27-08-734


.................................
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-29:17:32:33:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:32:33:INFO] No GPUs detected (normal if no gpus installed)
[2025-09-29:17:32:33:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this packag

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [14]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,842302,0.956720
1,844981,0.952197
2,846381,0.730796
3,849014,0.994905
4,8511133,0.956720
5,856106,0.964800
6,857374,0.004089
7,85922302,0.987876


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [15]:
sagemaker = boto3.client("sagemaker")

model_name = job_name
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2025-09-29-17-11-41
arn:aws:sagemaker:us-east-1:851725636446:model/xgb-2025-09-29-17-11-41


In [16]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2025-09-29-17-11-41',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:851725636446:training-job/xgb-2025-09-29-17-11-41',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-851725636446/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2025-09-29-17-11-41/xgb-2025-09-29-17-11-41/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

In [17]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:851725636446:endpoint-config/lab4-1-endpoint-config2025-09-29-17-33-13


In [18]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [19]:
# Wait for endpoint to spin up
from time import sleep

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [20]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.956719696521759



In [21]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '74e89fa8-3d3b-4b42-b0a0-d5a251e77ca4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '74e89fa8-3d3b-4b42-b0a0-d5a251e77ca4',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Mon, 29 Sep 2025 17:37:15 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '18',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7f2ee4932020>}

In [22]:
# Delete Endpoint

sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '09b9e2d1-7180-4d63-981d-94566a6662dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '09b9e2d1-7180-4d63-981d-94566a6662dc',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 29 Sep 2025 17:37:15 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)


# Part 1: Set Up Model Group

In [27]:
# Creating sagemaker client connection
sagemaker_client = boto3.client("sagemaker", region_name=region)

In [28]:
# Defining Model Group
model_package_group_name = "xgboost-breast-cancer-detection"
model_package_group_description = "Model group for versioned XGBoost models detecting breast cancer from WDBC dataset"

In [29]:
# Creating Model Package Group
response = sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription=model_package_group_description
)

print("Model Package Group ARN:", response["ModelPackageGroupArn"])

Model Package Group ARN: arn:aws:sagemaker:us-east-1:851725636446:model-package-group/xgboost-breast-cancer-detection


In [30]:
info = sagemaker_client.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

print(info)

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:851725636446:model-package-group/xgboost-breast-cancer-detection', 'ModelPackageGroupDescription': 'Model group for versioned XGBoost models detecting breast cancer from WDBC dataset', 'CreationTime': datetime.datetime(2025, 9, 29, 21, 4, 38, 620000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:851725636446:user-profile/d-ezz4yriklv7b/spaz', 'UserProfileName': 'spaz', 'DomainId': 'd-ezz4yriklv7b', 'IamIdentity': {'Arn': 'arn:aws:sts::851725636446:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROA4MTWN2NPN7WZQGXLO:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '433efb50-1b36-4b39-965a-ec002b382b69', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '433efb50-1b36-4b39-965a-ec002b382b69', 'strict-transport-security': 'max-age=47304000; includeSubDomains', 'x-frame-options': 'DENY', 'conten

# Part 2: Set Up Model Package

In [31]:
# Defining variables and primary container for the model
model_package_group_name = "xgboost-breast-cancer-detection"
model_name = job_name

primary_container = {
    "Image": image,
    "ModelDataUrl": model_data,
}

In [33]:
# Creating model package
response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="XGBoost breast cancer detection model, trained on WDBC dataset",
    InferenceSpecification={
        "Containers": [primary_container],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
        "SupportedRealtimeInferenceInstanceTypes": ["ml.m5.xlarge"],
        "SupportedTransformInstanceTypes": ["ml.m5.xlarge"]
    },
    ModelApprovalStatus="PendingManualApproval"
)

print("Model Package ARN:", response["ModelPackageArn"])

Model Package ARN: arn:aws:sagemaker:us-east-1:851725636446:model-package/xgboost-breast-cancer-detection/1


In [34]:
# Verifying model package
info = sagemaker_client.describe_model_package(
    ModelPackageName=response["ModelPackageArn"]
)

print(info)

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:851725636446:model-package/xgboost-breast-cancer-detection/1', 'ModelPackageDescription': 'XGBoost breast cancer detection model, trained on WDBC dataset', 'CreationTime': datetime.datetime(2025, 9, 29, 21, 16, 43, 75000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:b4f13edb198529c460692015797fa1ca6a8ff1ed64a149297174d922121b8fc4', 'ModelDataUrl': 's3://sagemaker-us-east-1-851725636446/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2025-09-29-17-11-41/xgb-2025-09-29-17-11-41/output/model.tar.gz', 'ModelDataETag': 'e83e0f9e81aa052b7fe142908ad6331c'}], 'SupportedTransformInstanceTypes': ['ml.m5.xlarge'], 'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.xlarge'], 'SupportedContentTypes': ['text/csv'], 'SupportedResponseMIME

# Part 3: Write the Model Card

In [52]:
# Preparing Model Card Content
import json

model_card_name = "xgboost-breast-cancer-model-card"

model_card_content = {
    "model_overview": {
        "model_description": "XGBoost model for breast cancer classification",
        "model_creator": "Sean Paz",
        "algorithm_type": "XGBoost",
        "problem_type": "Binary classification",
        "model_owner": "Sean Paz"
    },
    "intended_uses": {
        "purpose_of_model": "Predict whether a patient has breast cancer",
        "intended_uses": "Research and clinical decision support",
        "risk_rating": "Medium"
    },
    "business_details": {
        "business_problem": "Early detection of breast cancer"
    },
    "training_details": {
        "objective_function": {"function": "Maximize"},
        "training_job_details": {}
    },
    "evaluation_details": [],
    "additional_information": {}
}

# Converting dictionary to JSON string for create_model_card
model_card_json = json.dumps(model_card_content)

In [53]:
# Creating Model Card
response = sagemaker_client.create_model_card(
    ModelCardName=model_card_name,
    Content=model_card_json,
    ModelCardStatus="Draft"
)

print("Model Card ARN:", response["ModelCardArn"])

Model Card ARN: arn:aws:sagemaker:us-east-1:851725636446:model-card/xgboost-breast-cancer-model-card


In [54]:
# Verifying Model Card
info = sagemaker_client.describe_model_card(
    ModelCardName=model_card_name
)

print(info)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:851725636446:model-card/xgboost-breast-cancer-model-card', 'ModelCardName': 'xgboost-breast-cancer-model-card', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_description": "XGBoost model for breast cancer classification", "model_creator": "Sean Paz", "algorithm_type": "XGBoost", "problem_type": "Binary classification", "model_owner": "Sean Paz"}, "intended_uses": {"purpose_of_model": "Predict whether a patient has breast cancer", "intended_uses": "Research and clinical decision support", "risk_rating": "Medium"}, "business_details": {"business_problem": "Early detection of breast cancer"}, "training_details": {"objective_function": {"function": "Maximize"}, "training_job_details": {}}, "evaluation_details": [], "additional_information": {}}', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2025, 9, 29, 21, 50, 18, 459000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:8517256364